# Genetics

- PGS collected from PGS Catalog and mapping to same name standards of this study.
- Genotype used 1KG phase 3 data to impute.
- PGS constructed using plink2.



In [1]:
import pandas as pd
from pathlib import Path
from config import *
from utils import rsquareCI, rsquareCI_v2, R2difference

In [2]:
# Load PRS
PRS_df = pd.read_feather("Raw/PRS.feather")
PRS_df

,eid,PGS002783,PGS004208,PGS004207,PGS004206,PGS004210,PGS004209,PGS003834,PGS004676,PGS004642,...,PGS002991,PGS003974,PGS000311,PGS004850,PGS003127,PGS001405,PGS001229,PGS004005,PGS003227,PGS004715
0,9edbfe3d7e5e242978f03b61cee98b33,-1.195739,0.891136,0.859189,0.944204,0.907260,1.015556,-0.048539,0.092417,0.142837,...,-6.243971,-16.296718,0.299125,-0.554194,0.007288,-3.758395,1.340945,-0.036902,0.156812,-0.032258
1,275cd00e41e5a0c9c11547d43e0c8cd3,-1.454993,0.300677,0.260189,0.341992,0.314461,0.413811,0.012692,0.396092,0.075459,...,-4.801484,-27.173533,0.179500,-0.315302,0.002061,-3.056467,4.458528,0.021362,0.311969,0.045338
2,ab27d63a1a00a37f07ad43543334cf26,-0.757478,0.047295,0.025072,0.097761,0.052178,0.139525,0.170258,0.342630,0.198056,...,-6.272686,-17.656856,0.225375,-0.154875,0.001444,-3.159184,-3.363850,0.039912,0.176703,0.060803
3,c5c1076a4c44707a022680e14e6d766e,-0.403322,0.324868,0.305421,0.361992,0.338137,0.391710,0.201172,0.635398,0.080791,...,-5.428220,-11.072270,0.039030,-0.364455,0.006910,-4.071309,0.337697,0.062971,0.407423,0.069385
4,04650401f2ceab43bb253cef5dcf1f48,-0.584667,0.207730,0.177065,0.259714,0.215977,0.279726,0.050299,0.156356,0.125023,...,-4.746501,-16.248285,0.030888,-0.204775,0.005526,0.336435,0.956931,0.004833,0.102584,0.022154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,e5dceec66ae81d978673955c8853e142,-0.838441,-0.328531,-0.343099,-0.277572,-0.301508,-0.210605,0.184218,0.500042,0.215750,...,-4.678348,0.241843,0.320151,-0.200244,0.009253,-0.876307,0.122554,-0.032881,0.323427,0.046750
107,ca9314f170e65bba6ed7fa4e81385e39,-1.163888,-0.143207,-0.152926,-0.098200,-0.148996,-0.114854,0.127910,0.263748,0.089197,...,-4.745884,-12.397545,0.181205,-0.182357,0.008911,-2.033077,2.667862,-0.062020,0.261337,0.041371
108,3613fefdbb23400a654774414a213c25,-0.124342,0.310082,0.295585,0.339581,0.322347,0.331118,0.200945,0.229542,0.180233,...,-5.205582,-42.982657,0.031295,-0.114023,0.007118,-0.803844,4.449697,-0.075038,0.242041,0.046852
109,fa259145f00b760a566ad980003d2c27,-0.304229,0.070778,0.048873,0.122713,0.096442,0.169169,-0.060982,0.174661,0.124317,...,-3.512185,-18.716425,0.063103,-0.476709,0.004988,-1.740719,2.080136,0.000514,0.062980,0.114755


In [3]:
# Load clinical label
label_df = pd.read_feather("Raw/ClinicalLabels.feather").query(
    "DataSetName == 'AZ-GEV'"
)
label_df

,eid,DataSetName,AG,ALP,ALT,APTT,AST,Alb,BMI,BNP,...,UA,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex
9549,5baa9c383872ccb6ebb367abd7d10c66,AZ-GEV,NaN,NaN,16.0,33.0,20.0,NaN,28.38,44.0,...,409.0,4.93,5.30,83.0,NaN,2.6,NaN,NaN,58.0,Male
9550,cfe8caaa1e916c1d22ed43b7f06bff9a,AZ-GEV,NaN,NaN,28.0,30.9,18.0,NaN,28.29,8.0,...,470.9,4.59,6.33,100.0,NaN,0.9,NaN,NaN,41.0,Male
9551,086ccd244a863a74be42ca984c22b9d2,AZ-GEV,NaN,NaN,22.0,29.6,23.0,NaN,25.95,12.0,...,451.3,4.17,8.18,75.0,NaN,3.3,NaN,NaN,56.0,Male
9552,0c36a17f899a0d0ca25208b0d88ad74d,AZ-GEV,NaN,NaN,5.0,28.0,19.0,NaN,29.39,20.0,...,431.8,5.27,6.75,81.0,NaN,1.5,NaN,NaN,61.0,Male
9553,7bfe14a60b0bebbfbc04245a6f23afb9,AZ-GEV,NaN,NaN,24.0,34.7,29.0,NaN,25.55,38.0,...,583.1,4.31,7.08,63.0,NaN,1.1,NaN,NaN,65.0,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9655,d4bf28a280dd47f643f9b718845a6087,AZ-GEV,NaN,93.0,14.0,26.8,19.0,38.9,NaN,31.0,...,613.9,8.72,5.37,NaN,0.24,11.4,NaN,0.76,69.0,Male
9656,4e40c16a7f0d5d7ee03d1a87383e2491,AZ-GEV,NaN,103.0,14.0,30.4,16.0,46.4,NaN,56.0,...,254.4,4.14,8.68,NaN,1.16,6.6,NaN,0.23,61.0,Female
9657,62f6edff516405b023a24aacc337c78b,AZ-GEV,NaN,46.0,56.0,28.6,43.0,37.5,NaN,22.0,...,392.0,7.86,8.59,NaN,2.10,3.4,NaN,0.52,59.0,Male
9658,0f1b7aa42032e8d3484841af23bc37e4,AZ-GEV,NaN,48.0,20.0,32.2,15.0,42.1,NaN,59.0,...,367.5,7.59,6.62,NaN,0.46,4.7,NaN,0.50,69.0,Male


In [4]:
# Load Ummae predicted values
Ummae = pd.read_feather("Raw/Ummae.feather").query("DataSetName == 'AZ-GEV'")
Ummae

,eid,type,DataSetName,AG,ALP,ALT,APTT,AST,Alb,BMI,...,TSH,UA,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age
71447,74f6a96d528814da2024c5e028a1327d,test,AZ-GEV,16.238819,77.251459,28.729980,31.162631,20.751671,45.812930,26.414055,...,2.395996,333.911163,5.834947,6.481861,81.672272,1.511911,32.187611,2.784280,0.753516,59.192711
71448,74f6a96d528814da2024c5e028a1327d,test,AZ-GEV,16.208529,76.454933,30.329456,31.384714,21.532017,45.991810,26.165821,...,2.390811,329.314939,5.838350,6.544262,80.592522,1.948088,32.187611,2.865211,0.778098,58.803188
71449,74f6a96d528814da2024c5e028a1327d,test,AZ-GEV,16.350157,76.481644,30.446892,31.381540,20.994213,45.564755,26.177910,...,2.391527,337.758301,5.839227,6.489704,80.395630,1.933762,32.204121,2.825856,0.768468,59.290916
71450,74f6a96d528814da2024c5e028a1327d,test,AZ-GEV,16.172538,76.317154,29.911385,31.552380,20.745512,45.540420,26.320106,...,2.067998,332.106537,5.945011,6.706291,78.781116,1.952000,32.154587,2.912430,0.721887,58.064991
71451,74f6a96d528814da2024c5e028a1327d,test,AZ-GEV,15.855510,76.983528,23.579250,31.297672,20.585581,45.955029,26.509329,...,2.419592,327.097321,6.211244,6.672956,78.312347,1.927339,80.199342,2.704042,0.787852,58.803791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72281,26d5010ed5c4ec1672d0d277cc1c120d,test,AZ-GEV,15.890325,79.747393,20.387346,31.199767,20.060755,45.470493,26.637653,...,2.396576,337.248347,6.211351,6.506534,80.026739,1.940553,32.204121,2.904867,0.797387,66.768410
72282,26d5010ed5c4ec1672d0d277cc1c120d,test,AZ-GEV,16.016916,75.839165,21.310390,31.385273,20.101555,45.388786,26.922455,...,2.394700,332.117462,6.356379,6.447704,81.168212,1.934397,32.237141,2.930209,0.765391,64.678146
72283,26d5010ed5c4ec1672d0d277cc1c120d,test,AZ-GEV,15.906009,76.300285,22.159451,31.226845,19.917568,45.264301,26.869003,...,2.438484,344.733459,5.956304,6.410942,84.479622,1.978606,-80.456509,3.001332,0.768854,65.807365
72284,26d5010ed5c4ec1672d0d277cc1c120d,test,AZ-GEV,15.549671,76.277794,24.024031,31.228151,19.919876,45.476479,26.542487,...,2.408509,335.540802,6.073228,6.414882,83.393524,2.143486,32.220631,3.017615,0.802192,71.103938


In [5]:
PGS_traits_df = pd.read_csv("Raw/PGSCatalogID_To_Traits.csv")
PGS_traits = PGS_traits_df["Traits"].unique().tolist()
print(f"Total PGS traits: {len(PGS_traits)} with {len(PGS_traits_df)} PGSs")
PGS_traits_df

Total PGS traits: 26 with 679 PGSs


,Score,Traits
0,PGS000670,ALP
1,PGS004714,ALP
2,PGS000815,ALP
3,PGS003521,ALP
4,PGS001939,ALP
...,...,...
674,PGS003553,Urea
675,PGS004739,Urea
676,PGS004740,Urea
677,PGS004742,Urea


## Evaluate Performance

In [6]:
from ppp_prediction.corr_v3 import cal_corr_v3

compare_res_list = []
data_prs_lab_face_dict = {}
errors = []
for trait in PGS_traits:
    # for trait in ['HDLC']:
    try:
        df = (
            label_df[["eid", trait]]
            .merge(
                Ummae[["eid", trait]],
                on="eid",
                how="inner",
                suffixes=("_lab", "_ummae"),
            )
            .dropna()
            .groupby("eid")
            .mean()
        )  # get mean of ummae
    except:
        errors.append(trait)
        continue

    # merge prs

    need_prs_code = PGS_traits_df.query(f"Traits == '{trait}'")["Score"].tolist()
    need_prs_code = [col for col in need_prs_code if col in PRS_df.columns]  # check
    df = df.merge(
        PRS_df[["eid", *need_prs_code]].set_index("eid"),
        left_index=True,
        right_index=True,
    )
    df = df.dropna(axis=1, how="all")
    if df.columns.shape[0] == 2:
        continue
    metrics_qt = cal_corr_v3(
        df,
        x=df.columns[1:].tolist(),
        y=f"{trait}_lab",
        norm_x=True,
        # cov = cov
        model_type="glm",
    )
    compare_res_list.append(metrics_qt)
    data_prs_lab_face_dict[trait] = df

total 10 combination of  to cal by threads 4
ALP_lab ~ PGS000670
ALP_lab ~ PGS004714
ALP_lab ~ ALP_ummae
ALP_lab ~ PGS000815
ALP_lab ~ PGS003521
ALP_lab ~ PGS001939
ALP_lab ~ PGS004711
ALP_lab ~ PGS002157
ALP_lab ~ PGS004712
ALP_lab ~ PGS004713
total 4 combination of  to cal by threads 4
LPa_lab ~ LPa_ummae
LPa_lab ~ PGS002101
LPa_lab ~ PGS001888
LPa_lab ~ PGS000671
total 11 combination of  to cal by threads 4
AST_lab ~ AST_ummae
AST_lab ~ PGS004721
AST_lab ~ PGS004722
AST_lab ~ PGS004719
AST_lab ~ PGS003523
AST_lab ~ PGS003352
AST_lab ~ PGS002159
AST_lab ~ PGS004330
AST_lab ~ PGS001941AST_lab ~ PGS000673

AST_lab ~ PGS003343
total 12 combination of  to cal by threads 4
T_Bil_lab ~ T_Bil_ummae
T_Bil_lab ~ PGS004209
T_Bil_lab ~ PGS004731
T_Bil_lab ~ PGS004207
T_Bil_lab ~ PGS002160
T_Bil_lab ~ PGS000681
T_Bil_lab ~ PGS004732
T_Bil_lab ~ PGS003524
T_Bil_lab ~ PGS004208
T_Bil_lab ~ PGS004206
T_Bil_lab ~ PGS004210
T_Bil_lab ~ PGS001942
total 91 combination of  to cal by threads 4
Height_lab

In [7]:
from ppp_prediction.corr_v3 import generate_multipletests_result

# 1) fdr_bh
compare_res_df_fdr = (
    generate_multipletests_result(
        pd.concat(compare_res_list).dropna(subset="pvalue"), method="fdr_bh"
    )
    .sort_values("r2_score", ascending=False)[
        ["x", "y", "pearsonr", "spearmanr", "N", "r2_score", "p_adj", "pvalue"]
    ]
    .rename(columns={"r2_score": "R2"})
)

compare_res_df_fdr = compare_res_df_fdr.rename(columns={"x": "Feature", "y": "Traits"})
compare_res_df_fdr["Traits"] = compare_res_df_fdr["Traits"].str.split("_").str[0]
compare_res_df_fdr["Feature"] = compare_res_df_fdr["Feature"].apply(
    lambda x: "Face" if "ummae" in x else x
)
# 2) add 95% CI and format the metrics
from ppp_prediction.metrics.utils import format_metrics

se, lci, uci = rsquareCI_v2(compare_res_df_fdr["R2"], compare_res_df_fdr["N"], 1)
compare_res_df_fdr["R2_SE"] = se
compare_res_df_fdr["R2_LCI"] = lci
compare_res_df_fdr["R2_UCI"] = uci

add_metric = ["R2"]

for metric in add_metric:

    compare_res_df_fdr[f"{metric} (95% CI)"] = format_metrics(
        compare_res_df_fdr[f"{metric}"],
        x_lci=compare_res_df_fdr[f"{metric}_LCI"],
        x_uci=compare_res_df_fdr[f"{metric}_UCI"],
        round=3,
        data=compare_res_df_fdr,
    )
# 3) assign ScoreType
compare_res_df_fdr["ScoreType"] = compare_res_df_fdr["Feature"].apply(
    lambda x: "Face" if x == "Face" else "PRS"
)
# 4) save the result
compare_res_df_fdr = compare_res_df_fdr.reset_index(drop=True)
compare_res_df_fdr.to_feather(RawDataDir / "Genetics_120_results.feather")
compare_res_df_fdr

,Feature,Traits,pearsonr,spearmanr,N,R2,p_adj,pvalue,R2_SE,R2_LCI,R2_UCI,R2 (95% CI),ScoreType
0,Face,Weight,0.709594,0.727165,66,5.035242e-01,1.843660e-13,7.845361e-16,0.082262,0.339000,0.668048,"0.504 (0.339, 0.668)",Face
1,Face,Height,0.681697,0.658352,66,4.647110e-01,1.596410e-11,9.057647e-14,0.085206,0.294299,0.635123,"0.465 (0.294, 0.635)",Face
2,Face,HCT,0.640139,0.644037,111,4.097774e-01,1.248068e-15,3.342308e-18,0.069501,0.270776,0.548779,"0.410 (0.271, 0.549)",Face
3,Face,Hb,0.639854,0.657141,111,4.094134e-01,1.248068e-15,3.540618e-18,0.069513,0.270388,0.548439,"0.409 (0.270, 0.548)",Face
4,Face,BMI,0.594923,0.662826,66,3.539328e-01,3.218344e-07,3.195519e-09,0.089749,0.174435,0.533431,"0.354 (0.174, 0.533)",Face
...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,PGS001405,Height,0.002861,-0.031840,66,8.186989e-06,9.873396e-01,9.817377e-01,0.000668,-0.001328,0.001344,"0.000 (-0.001, 0.001)",PRS
701,PGS001229,Height,0.001234,0.046911,66,1.523536e-06,9.949439e-01,9.921214e-01,0.000288,-0.000575,0.000578,"0.000 (-0.001, 0.001)",PRS
702,PGS004005,eGFR(CKD-EPI),0.000951,0.007788,111,9.038214e-07,9.949439e-01,9.920807e-01,0.000175,-0.000349,0.000351,"0.000 (-0.000, 0.000)",PRS
703,PGS003227,eGFR(CKD-EPI),0.000765,-0.039321,111,5.854793e-07,9.950375e-01,9.936261e-01,0.000141,-0.000281,0.000282,"0.000 (-0.000, 0.000)",PRS


## Calculate Relative importance

In [8]:
import re
import string
from itertools import product
from typing import Dict, List, Union

import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from pandas import DataFrame

from ppp_prediction.ci import bootstrap_ci
from ppp_prediction.norm import rank_INT


def get_cat_var_name(x):
    if x.startswith("C("):
        return re.findall(r"\((.*?)\)", x)[0]
    else:
        return x


def get_cat_var_subname(x):
    # match [*]
    try:
        return re.findall(r"\[(.*?)\]", x)[0].split(".")[1]
    except:
        return x


class columnsFormatV1:
    """
    format columns of df to remove space

    use format to remove

    use reverse to get original column name from formatted column name
    """

    def __init__(self, data):
        self.data = data
        self.columns = data.columns

        self.special_chars = "≥≤·！@#￥%……&*（）—+，。？、；：“”‘’《》{}【】 ><+-(),.//"
        self.columns_dict = {
            i: re.sub(
                f"[{re.escape(self.special_chars)}]",
                "_",
                i,
                # i.translate(str.maketrans("", "", string.punctuation)).replace(
                #     " ", "_"
                # ),
            )
            for i in self.columns
        }
        self.columns_dict_reverse = {v: k for k, v in self.columns_dict.items()}

    def format(self, data):
        return data.rename(columns=self.columns_dict)

    def reverse(self, data):
        return data.rename(columns=self.columns_dict_reverse)

    def get_format_column(self, column):
        if isinstance(column, list):
            return [self.columns_dict.get(i, i) for i in column]
        return self.columns_dict.get(column, column)

    def get_reverse_column(self, column):
        if isinstance(column, list):
            return [self.columns_dict_reverse.get(i, i) for i in column]

        return self.columns_dict_reverse.get(column, column)

    def __str__(self):
        return f"columnsFormat: {self.columns_dict}"

    def __repr__(self):
        return self.__str__()


def calc_relimp(
    data,
    label=None,
    vars=None,
    method="lmg",
    rela=True,
    cat_cols=None,
    binary_y=False,
    formula=None,
):

    import rpy2.robjects as robjects
    from rpy2.robjects import pandas2ri

    # define the R NULL object
    r_null = robjects.r("NULL")

    # 启用 R 和 pandas 数据转换功能
    pandas2ri.activate()

    # load local package
    # TODO: install R package code
    robjects.r(
        """
        options(warn = -1)
        library(relaimpo)


               """
    )

    calc_relimp_R = robjects.r(
        """

function(
  formula, data,type="lmg",rela=T,cat_cols=NULL, binary_y=F
){
    if (is.null(cat_cols)){
    for (col in cat_cols){
        data[[col]] <- as.factor(data[[col]])
    }
    }




  fit <- glm(formula, data = data)
  crf<-calc.relimp(fit, type = type,rela=rela)
  res <- as.data.frame(
    rbind(
      c(slot(crf,type), slot(crf,"R2.decomp")))
  )
  length <- length(res)
  colnames(res) <- c(colnames(res)[1:length-1], "R2.decomp")

  if (binary_y){
  logit.fit <- glm(formula, data = data, family = binomial(link = "logit"))
  R2 <- 1 - logit.fit$deviance / logit.fit$null.deviance
    res$R2.decomp <- R2
  }
  return (res)
    }
  


        """
    )
    # format the data

    if formula is None and label is not None and vars is not None:
        formula = label + "~" + "+".join(vars)
    elif formula is None:
        pass
    else:
        raise ValueError("Please provide formula or label and vars")
    res = calc_relimp_R(
        formula=formula,
        data=data,
        type=method,
        rela=rela,
        binary_y=robjects.BoolVector([binary_y]),
        cat_cols=robjects.StrVector(cat_cols) if cat_cols else r_null,
    )

    res = pandas2ri.rpy2py(res).reset_index(drop=True)

    return res

In [9]:
# each traits only keep the best performance PRS
face_verse_prs_df = (
    compare_res_df_fdr.sort_values("R2", ascending=False)
    .groupby(["Traits", "ScoreType"])
    .head(1)
    .sort_values("R2", ascending=False)
)
# order = face_verse_prs_df.query("ScoreType == 'Face'")['Traits'].drop_duplicates().values.tolist()
order = (
    (
        face_verse_prs_df.query("ScoreType == 'Face'")
        .set_index("Traits")["R2"]
        .sort_index()
        - face_verse_prs_df.query("ScoreType == 'PRS'")
        .sort_values("Traits")
        .set_index("Traits")["R2"]
        .sort_index()
    )
    .sort_values(ascending=False)
    .index.tolist()
)
face_verse_prs_df["Traits"] = pd.Categorical(
    face_verse_prs_df["Traits"], categories=order, ordered=True
)
face_verse_prs_df

,Feature,Traits,pearsonr,spearmanr,N,R2,p_adj,pvalue,R2_SE,R2_LCI,R2_UCI,R2 (95% CI),ScoreType
0,Face,Weight,0.709594,0.727165,66,0.503524,1.843660e-13,7.845361e-16,0.082262,0.339000,0.668048,"0.504 (0.339, 0.668)",Face
1,Face,Height,0.681697,0.658352,66,0.464711,1.596410e-11,9.057647e-14,0.085206,0.294299,0.635123,"0.465 (0.294, 0.635)",Face
2,Face,HCT,0.640139,0.644037,111,0.409777,1.248068e-15,3.342308e-18,0.069501,0.270776,0.548779,"0.410 (0.271, 0.549)",Face
3,Face,Hb,0.639854,0.657141,111,0.409413,1.248068e-15,3.540618e-18,0.069513,0.270388,0.548439,"0.409 (0.270, 0.548)",Face
4,Face,BMI,0.594923,0.662826,66,0.353933,3.218344e-07,3.195519e-09,0.089749,0.174435,0.533431,"0.354 (0.174, 0.533)",Face
5,Face,HDLC,0.564992,0.519304,111,0.319215,1.231761e-10,8.735893e-13,0.070754,0.177708,0.460723,"0.319 (0.178, 0.461)",Face
6,Face,Cr,0.493614,0.438528,111,0.243655,3.218344e-07,3.109393e-09,0.068676,0.106303,0.381007,"0.244 (0.106, 0.381)",Face
7,PGS002783,TC,0.462130,0.385908,111,0.213565,4.679723e-06,5.310324e-08,0.066854,0.079857,0.347272,"0.214 (0.080, 0.347)",PRS
8,Face,eGFR(CKD-EPI),0.434329,0.411492,111,0.188642,3.759892e-05,4.799862e-07,0.064823,0.058995,0.318288,"0.189 (0.059, 0.318)",Face
9,PGS004208,T-Bil,0.426643,0.373984,111,0.182025,5.557906e-05,8.435078e-07,0.064195,0.053634,0.310415,"0.182 (0.054, 0.310)",PRS


In [10]:
# calculate relative importance of face and genetics

RelativeDataDir = RawDataDir / "RelativeImportance_Cardiovascular_full_Genetics.csv"


if not (RelativeDataDir).exists():
    res_relimp_list = []
    for trait, subdf in face_verse_prs_df.groupby("Traits"):
        prs_col = subdf.query("ScoreType == 'PRS'")["Feature"].values[0]
        face_col = f"{trait}_ummae"
        lab_col = f"{trait}_lab"
        omics = [prs_col, face_col]

        RelativeImportanceData = data_prs_lab_face_dict[trait][omics + [lab_col]].copy()

        RelativeImportanceDataFromat = columnsFormatV1(RelativeImportanceData)

        formated_data = RelativeImportanceDataFromat.format(RelativeImportanceData)
        omics_format = [
            RelativeImportanceDataFromat.get_format_column(i) for i in omics
        ]
        label_col_format = RelativeImportanceDataFromat.get_format_column(lab_col)

        method = "lmg"
        relimp_df_Ummae_full = (
            calc_relimp(
                formated_data,
                label=label_col_format,
                vars=omics_format,
                method=method,
                binary_y=False,
            )
            .assign(
                Method=method,
            )
            .melt(
                id_vars=["Method", "R2.decomp"],
                value_vars=omics_format,
                var_name="Omics",
                value_name="Importance",
            )
        )
        relimp_df_Ummae_full["Traits"] = trait
        print(RelativeImportanceDataFromat, face_col)
        relimp_df_Ummae_full["Omics"] = relimp_df_Ummae_full["Omics"].replace(
            {
                RelativeImportanceDataFromat.get_reverse_column(face_col): "Face",
                RelativeImportanceDataFromat.get_reverse_column(prs_col): "Genetics",
            }
        )
        relimp_df_Ummae_full["Omics"] = relimp_df_Ummae_full["Omics"].apply(
            lambda x: "Face" if "ummae" in x else x
        )
        res_relimp_list.append(relimp_df_Ummae_full)
        # break

    relim_df = pd.concat(res_relimp_list)
    relim_df.to_csv(RelativeDataDir, index=False)
else:
    relim_df = pd.read_csv(RelativeDataDir)